In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time

In [14]:
response = requests.get("https://www.balldontlie.io/api/v1/stats?seasons[]=2020&per_page=100").json()
response['data'][0]

{'id': 2848769,
 'ast': 3,
 'blk': 1,
 'dreb': 4,
 'fg3_pct': 50.0,
 'fg3a': 2,
 'fg3m': 1,
 'fg_pct': 43.8,
 'fga': 16,
 'fgm': 7,
 'ft_pct': 100.0,
 'fta': 7,
 'ftm': 7,
 'game': {'id': 127502,
  'date': '2020-12-22T00:00:00.000Z',
  'home_team_id': 3,
  'home_team_score': 125,
  'period': 4,
  'postseason': False,
  'season': 2020,
  'status': 'Final',
  'time': '',
  'visitor_team_id': 10,
  'visitor_team_score': 99},
 'min': '24:56',
 'oreb': 1,
 'pf': 3,
 'player': {'id': 140,
  'first_name': 'Kevin',
  'height_feet': 6,
  'height_inches': 9,
  'last_name': 'Durant',
  'position': 'F',
  'team_id': 3,
  'weight_pounds': 240},
 'pts': 22,
 'reb': 5,
 'stl': 3,
 'team': {'id': 3,
  'abbreviation': 'BKN',
  'city': 'Brooklyn',
  'conference': 'East',
  'division': 'Atlantic',
  'full_name': 'Brooklyn Nets',
  'name': 'Nets'},
 'turnover': 1}

In [15]:
# function to loop through all pages and extract data of interest for each player for the season
def get_season_stats(season):
    
    base_url = "https://www.balldontlie.io/api/v1/stats"
    query = f"?seasons[]={season}&per_page=100&postseason=false"

    response = requests.get(base_url + query).json()

    stats = {}
    page = 1
    index = 0
    total_count = response['meta']['total_count']
    total_pages = response['meta']['total_pages']
    
    print("-------------------------------------------------")
    print(f"collecting data for the {season} season...")
    print("-------------------------------------------------")

    for i in range(total_count):
        # go to next page when index is 100 since there are 100 entries per page
        if index == 100:
            page += 1
            
            # pause in order to not overload the API
            if page % 60 == 0:
                print("taking a breather...")
                time.sleep(20)
            
            # reset index and call API to get next page
            index = 0
            query = f"?seasons[]={season}&page={page}&per_page=100&postseason=false"
            response = requests.get(base_url + query).json()
            
            # status update
            if page % 10 == 0:
                print(f"currently on page {page} out of {total_pages}")
                
        try:
            # extract assists, points, field goal attempts, field goals made,
            # free throw attempts, free throws made, and player id
            ast = response['data'][index]['ast']
            pts = response['data'][index]['pts']
            fga = response['data'][index]['fga']
            fgm = response['data'][index]['fgm']
            fta = response['data'][index]['fta']
            ftm = response['data'][index]['ftm']
            player_id = response['data'][index]['player']['id']
            
            # add player to dictionary and initialize stats if new
            # need to check for None type
            if player_id not in stats:
                stats[player_id] = {}
                stats[player_id]['first_name'] = response['data'][index]['player']['first_name']
                stats[player_id]['last_name'] = response['data'][index]['player']['last_name']
                if ast:
                    stats[player_id]['ast'] = ast
                else: 
                    stats[player_id]['ast'] = 0
                if pts:
                    stats[player_id]['pts'] = pts
                else:
                    stats[player_id]['pts'] = 0
                if fga:
                    stats[player_id]['fga'] = fga
                else:
                    stats[player_id]['fga'] = 0
                if fgm:
                    stats[player_id]['fgm'] = fgm
                else: 
                    stats[player_id]['fgm'] = 0
                if fta:
                    stats[player_id]['fta'] = fta
                else:
                    stats[player_id]['fta'] = 0
                if ftm:
                    stats[player_id]['ftm'] = ftm
                else:
                    stats[player_id]['ftm'] = 0
                stats[player_id]['games'] = 1
            
            # update values if player is already in dict
            else:
                if ast: stats[player_id]['ast'] += ast
                if pts: stats[player_id]['pts'] += pts
                if fga: stats[player_id]['fga'] += fga
                if fgm: stats[player_id]['fgm'] += fgm
                if fta: stats[player_id]['fta'] += fta
                if ftm: stats[player_id]['ftm'] += ftm  
                stats[player_id]['games'] += 1

        except:
            print(f"Something went wrong page: {page}, index: {index}")
            print("skipping...")

        index += 1
        
    return stats

In [3]:
# Initialize dict to carry all data that'll eventually go to our database and be called
player_stats = {}

In [9]:
# Populate stats
# WARNING! This may take a while...
for season in range(2011, 2021):
    player_stats[season] = get_season_stats(season)
    time.sleep(20)

-------------------------------------------------
collecting data for the 2011 season...
-------------------------------------------------
currently on page 10 out of 255
currently on page 20 out of 255
currently on page 30 out of 255
currently on page 40 out of 255
currently on page 50 out of 255
taking a breather...
currently on page 60 out of 255
currently on page 70 out of 255
currently on page 80 out of 255
currently on page 90 out of 255
currently on page 100 out of 255
currently on page 110 out of 255
taking a breather...
currently on page 120 out of 255
currently on page 130 out of 255
currently on page 140 out of 255
currently on page 150 out of 255
currently on page 160 out of 255
currently on page 170 out of 255
taking a breather...
currently on page 180 out of 255
currently on page 190 out of 255
currently on page 200 out of 255
currently on page 210 out of 255
currently on page 220 out of 255
currently on page 230 out of 255
taking a breather...
currently on page 240 out o

currently on page 220 out of 310
currently on page 230 out of 310
taking a breather...
currently on page 240 out of 310
currently on page 250 out of 310
currently on page 260 out of 310
currently on page 270 out of 310
currently on page 280 out of 310
currently on page 290 out of 310
taking a breather...
currently on page 300 out of 310
currently on page 310 out of 310
-------------------------------------------------
collecting data for the 2018 season...
-------------------------------------------------
Something went wrong page: 8, index: 31
skipping...
currently on page 10 out of 309
currently on page 20 out of 309
currently on page 30 out of 309
currently on page 40 out of 309
currently on page 50 out of 309
taking a breather...
currently on page 60 out of 309
currently on page 70 out of 309
currently on page 80 out of 309
currently on page 90 out of 309
currently on page 100 out of 309
currently on page 110 out of 309
taking a breather...
currently on page 120 out of 309
currentl

In [12]:
player_stats[2020]

{140: {'first_name': 'Kevin',
  'last_name': 'Durant',
  'ast': 186,
  'pts': 910,
  'fga': 581,
  'fgm': 313,
  'fta': 229,
  'ftm': 201,
  'games': 41},
 188: {'first_name': 'Jeff',
  'last_name': 'Green',
  'ast': 107,
  'pts': 745,
  'fga': 527,
  'fgm': 259,
  'fta': 160,
  'ftm': 125,
  'games': 69},
 250: {'first_name': 'DeAndre',
  'last_name': 'Jordan',
  'ast': 93,
  'pts': 421,
  'fga': 248,
  'fgm': 188,
  'fta': 90,
  'ftm': 45,
  'games': 70},
 115: {'first_name': 'Stephen',
  'last_name': 'Curry',
  'ast': 357,
  'pts': 1984,
  'fga': 1348,
  'fgm': 649,
  'fta': 387,
  'ftm': 354,
  'games': 69},
 228: {'first_name': 'Kyrie',
  'last_name': 'Irving',
  'ast': 318,
  'pts': 1426,
  'fga': 1059,
  'fgm': 539,
  'fta': 216,
  'ftm': 199,
  'games': 54},
 468: {'first_name': 'Brad',
  'last_name': 'Wanamaker',
  'ast': 172,
  'pts': 327,
  'fga': 294,
  'fgm': 113,
  'fta': 94,
  'ftm': 85,
  'games': 72},
 36: {'first_name': 'Kent',
  'last_name': 'Bazemore',
  'ast': 108,

In [13]:
# export data to json file
import json

with open("player_stats.json", "w") as f:
    json.dump(player_stats, f)